<a href="https://colab.research.google.com/github/ErdemAslans/MultiDisease-Analysis-Platform/blob/main/Multidisease_Anaylsis_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install  segmentation_models_pytorch
!pip install monai
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.5 MB/s eta 0:00:00


In [ ]:
!pip install scikit-image

In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.2 MB/s eta 0:00:00


In [ ]:
!pip install bm3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.0/862.0 kB 38.4 MB/s eta 0:00:00


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install face_alignment

In [ ]:
!pip install onnxruntime
!pip install deepface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=3a7d7e64f0f2cb9c5e420243b9a4f7198e8e76b5cd1e7d5231601612f59b8627
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
!pip install facenet-pytorch

In [ ]:
import cv2
import os
import torch
import traceback
import gradio as gr
import numpy as np
import nibabel as nib
import pywt
from skimage.transform import resize
from skimage.restoration import denoise_tv_chambolle

from facenet_pytorch import MTCNN
from transformers import ViTFeatureExtractor as HuggingFaceVitFeatureExtractor
from transformers import ViTModel, SwinModel, SwinConfig
import onnxruntime as ort

# Albumentations
from albumentations import (Compose, HorizontalFlip, RandomBrightnessContrast, Rotate,
                            CLAHE, CoarseDropout, MotionBlur, GaussNoise, Perspective,
                            Resize, Sharpen, RandomGamma, ElasticTransform)
from albumentations.pytorch import ToTensorV2

import timm
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

# ---------------------------------------------------
# 0) Ortak Ayarlar
# ---------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

base_path = '/content/drive/My Drive/face_recognition'
os.makedirs(base_path, exist_ok=True)
faces_path = os.path.join(base_path, 'known_faces.pth')

# ---------------------------------------------------
# 1) Yüz Tanıma Modeli & Bilinen Yüzler
# ---------------------------------------------------
# MTCNN yüz tespiti
mtcnn = MTCNN(keep_all=True, device=device)
print("MTCNN model yüklendi.")

# ArcFace ONNX yükle
arcface_model_path = '/content/drive/My Drive/face_recognition/webface_r50.onnx'
ort_session = ort.InferenceSession(arcface_model_path)
print("ArcFace ONNX modeli yüklendi.")

# ViT ve Swin model (örnek)
vit_feature_extractor = HuggingFaceVitFeatureExtractor.from_pretrained('google/vit-large-patch16-224')
vit_model = ViTModel.from_pretrained('google/vit-large-patch16-224').eval().to(device)

swin_config = SwinConfig.from_pretrained('microsoft/swin-large-patch4-window7-224')
swin_model = SwinModel(swin_config).eval().to(device)

# Bilinen yüz embeddingleri yükle
try:
    known_faces = torch.load(faces_path)
    print(f"Kayıtlı yüzler yüklendi. Toplam kayıtlı yüz sayısı: {len(known_faces)}")
except FileNotFoundError:
    known_faces = {}
    print("Kayıtlı yüz bulunamadı. Yeni bir sözlük oluşturuluyor...")

# Veri artırma
augmentation = Compose([
    HorizontalFlip(p=0.3),
    RandomBrightnessContrast(p=0.4),
    Rotate(limit=20, p=0.3),
    CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.3),
    CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.3),
    MotionBlur(blur_limit=7, p=0.2),
    GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    Perspective(scale=(0.05, 0.1), p=0.3),
    Resize(224, 224),
    Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.3),
    RandomGamma(gamma_limit=(80, 120), p=0.3),
    ElasticTransform(alpha=1.0, sigma=50.0, p=0.2),
    ToTensorV2()
])

def extract_vit_swin_features(model, augmented):
    with torch.no_grad():
        vit_output = model(augmented.unsqueeze(0).float())
        if hasattr(vit_output, 'last_hidden_state'):
            features = vit_output.last_hidden_state[:, 0, :]  # CLS token
        else:
            features = vit_output.pooler_output
        features = torch.nn.functional.adaptive_avg_pool1d(features.unsqueeze(0), 1).squeeze(0)
    return features

def extract_face_embedding(frame):
    """
    Mevcut kodunuzdaki yüz tespiti + embedding çıkarma
    (ArcFace + ViT + Swin birleştirme) fonksiyonu.
    """
    try:
        faces, _ = mtcnn.detect(frame)
        if faces is None or len(faces) == 0:
            return None, None, "Yüz tespit edilmedi!"

        x1, y1, x2, y2 = map(int, faces[0])
        if x2 - x1 <= 0 or y2 - y1 <= 0:
            return None, None, "Geçersiz yüz koordinatları!"

        face = frame[y1:y2, x1:x2]
        if face is None or face.size == 0:
            return None, None, "Geçersiz yüz kırpma!"

        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_resized = cv2.resize(face_rgb, (112, 112))
        face_resized = (face_resized.astype(np.float32) - 127.5) / 128.0
        face_resized = np.transpose(face_resized, (2, 0, 1))
        face_resized = np.expand_dims(face_resized, axis=0)

        # ArcFace
        input_name = ort_session.get_inputs()[0].name
        output_name = ort_session.get_outputs()[0].name
        arcface_out = ort_session.run([output_name], {input_name: face_resized.astype(np.float32)})[0]
        arcface_out = arcface_out / np.linalg.norm(arcface_out, axis=1, keepdims=True)
        arcface_out = torch.tensor(arcface_out).to(device)

        # ViT & Swin Embedding
        augmented = augmentation(image=face_rgb)['image'].to(device)
        vit_features = extract_vit_swin_features(vit_model, augmented)
        swin_features = extract_vit_swin_features(swin_model, augmented)

        # Birleştirme
        combined_features = torch.cat((vit_features, swin_features, arcface_out), dim=1)
        combined_features = torch.nn.functional.normalize(combined_features, p=2, dim=1)

        return combined_features, faces, "Embedding çıkarıldı."
    except Exception as e:
        return None, None, f"Hata: {str(e)}"

def calculate_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=1).item()

def login_with_face(frame, session_state):
    """
    Kullanıcı 'Login' sekmesinde yüz tanıma yapar.
    Eğer tanınırsa session_state["token"] = "AUTHORIZED" olarak ayarlanır.
    """
    if frame is None:
        return None, "Resim alınamadı!", session_state

    embedding, faces, msg = extract_face_embedding(frame)
    if embedding is None:
        session_state["token"] = None
        return frame, "Login başarısız: " + msg, session_state

    best_match = None
    highest_similarity = 0.0
    threshold = 0.28

    if not known_faces:
        session_state["token"] = None
        return frame, "Hiç kayıtlı yüz yok!", session_state

    # Mevcut kayıtlı yüzlerle kıyasla
    for name, known_embedding in known_faces.items():
        if embedding.shape != known_embedding.shape:
            # Boyut uyuşmazlığı varsa geç
            continue
        sim = calculate_similarity(embedding, known_embedding.to(device))
        if sim > highest_similarity:
            highest_similarity = sim
            best_match = name

    if best_match is not None and highest_similarity > threshold:
        # Login başarılı
        session_state["token"] = "AUTHORIZED"
        msg_success = f"Giriş Başarılı! Tanınan: {best_match}, Benzerlik: {highest_similarity:.2f}"
        return frame, msg_success, session_state
    else:
        # Login başarısız
        session_state["token"] = None
        msg_fail = f"Giriş reddedildi! En yüksek benzerlik: {highest_similarity:.2f}"
        return frame, msg_fail, session_state

def add_new_face(name, frame):
    """
    Yüz kaydetme işlevi (mevcut kodunuzdaki gibi).
    """
    if frame is None or name.strip() == "":
        return frame, "Geçersiz girdi!"

    embedding, faces, msg = extract_face_embedding(frame)
    if embedding is None:
        return frame, "Embedding çıkarılamadı: " + msg

    if name in known_faces:
        return frame, f"{name} zaten kayıtlı!"
    known_faces[name] = embedding.cpu()
    torch.save(known_faces, faces_path)
    return frame, f"{name} sisteme eklendi ve kaydedildi."

# ---------------------------------------------------
# 2) Meme Kanseri Segmentasyon ve PH Analizi Modelleri
# ---------------------------------------------------
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAMBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        ca = self.channel_attention(x)
        x = x * ca
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        sa = torch.cat([avg_out, max_out], dim=1)
        sa = self.spatial_attention(sa)
        x = x * sa
        return x

class MultiScaleFeatureFusion(nn.Module):
    def __init__(self, embed_dim, num_classes=1, dropout_rate=0.1):
        super(MultiScaleFeatureFusion, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv_pred = nn.Conv2d(embed_dim // 2, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)

        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        x1 = F.interpolate(self.conv1(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        x2 = self.conv2(x)
        x3 = F.interpolate(self.conv3(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        fused = x1 + x2 + x3
        fused = self.dropout(fused)
        output = self.conv_pred(fused)
        return output

class ViTFeatureExtractor(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViTFeatureExtractor, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, drop_rate=dropout_rate, drop_path_rate=0.2)
        self.vit.head = nn.Identity()
        self.embed_dim = self.vit.embed_dim

    def forward(self, x):
        x = self.vit.forward_features(x)
        return x

class SegFormerDecoderWithAttention(nn.Module):
    def __init__(self, embed_dim, num_classes=1, patch_size=16, dropout_rate=0.4):
        super(SegFormerDecoderWithAttention, self).__init__()
        self.patch_size = patch_size
        self.num_patches_side = 224 // self.patch_size
        self.linear_fuse = nn.Linear(embed_dim, embed_dim)
        self.conv_pred = nn.Conv2d(embed_dim, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.cbam = CBAMBlock(embed_dim)
        self.multi_scale_fusion = MultiScaleFeatureFusion(embed_dim, num_classes, dropout_rate)
        nn.init.kaiming_normal_(self.linear_fuse.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        B, N, C = x.shape
        x = x[:, 1:, :]
        x = self.linear_fuse(x)
        x = x.transpose(1, 2).contiguous().view(B, C, self.num_patches_side, self.num_patches_side)
        x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        x = self.cbam(x)
        x = self.multi_scale_fusion(x)
        return x

class ViT_SegFormer_Model(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViT_SegFormer_Model, self).__init__()
        self.vit = ViTFeatureExtractor(dropout_rate)
        self.decoder = SegFormerDecoderWithAttention(embed_dim=self.vit.embed_dim, dropout_rate=dropout_rate)

    def forward(self, x):
        features = self.vit(x)
        output = self.decoder(features)
        return output

# Meme kanseri segmentasyon modeli
tumor_model = ViT_SegFormer_Model(dropout_rate=0.2).to(device)
tumor_weight_path = "/content/drive/MyDrive/Tümör/best_model.pth"
try:
    state_dict_tumor = torch.load(tumor_weight_path, map_location=device)
    tumor_model.load_state_dict(state_dict_tumor)
    tumor_model.eval()
    print("Breast Cancer Segmentation model loaded successfully.")
except Exception as e:
    print(f"Error loading Breast Cancer Segmentation model: {e}")

# Pulmoner Hipertansiyon Modeli
class CustomViT(nn.Module):
    def __init__(self, pretrained_vit, num_classes):
        super(CustomViT, self).__init__()
        self.vit = pretrained_vit
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(pretrained_vit.heads.head.in_features, num_classes)
        self.vit.heads.head = self.fc

    def forward(self, x):
        x = self.vit(x)
        x = self.dropout(x)
        return x

pretrained_vit = torchvision.models.vit_b_16(
    weights=torchvision.models.ViT_B_16_Weights.DEFAULT
).to(device)
pulmonary_model = CustomViT(pretrained_vit, num_classes=4).to(device)
pulmonary_weight_path = "/content/drive/MyDrive/Pulmoner Hipertansiyon/best_model.pth"
try:
    state_dict_pulmonary = torch.load(pulmonary_weight_path, map_location=device)
    pulmonary_model.load_state_dict(state_dict_pulmonary)
    pulmonary_model.eval()
    print("Pulmonary Hypertension Analysis model loaded successfully.")
except Exception as e:
    print(f"Error loading Pulmonary Hypertension Analysis model: {e}")

# ---------------------------------------------------
# Medikal İşleme Fonksiyonları
# ---------------------------------------------------
def enhance_contrast(image, clip_limit=0.6, tile_grid_size=(32,32), gamma=0.3):
    import cv2
    min_val = np.min(image)
    max_val = np.max(image)
    if max_val - min_val == 0:
        image = np.zeros_like(image)
    else:
        image = (image - min_val) / (max_val - min_val)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    image = clahe.apply((image * 255).astype(np.uint8)) / 255.0

    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    image = cv2.LUT((image * 255).astype(np.uint8), table).astype(np.float32) / 255

    laplacian = cv2.Laplacian(image.astype(np.float64), cv2.CV_64F)
    laplacian = np.clip(laplacian, 0, 1)
    laplacian_weight = 0.33
    enhanced_image = image + laplacian_weight * laplacian
    enhanced_image = np.clip(enhanced_image, 0, 1)
    return enhanced_image

def apply_nlm_denoising(image):
    import cv2
    denoised_image = cv2.fastNlMeansDenoising(np.uint8(image * 255), None,
                                              h=10, templateWindowSize=7, searchWindowSize=21)
    denoised_image = denoised_image.astype(np.float32) / 255.0
    return denoised_image

def apply_wavelet_denoising(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    coeffs = list(coeffs)
    coeffs[1:] = [tuple(pywt.threshold(c, value=0.1, mode='soft') for c in level)
                  for level in coeffs[1:]]
    denoised_image = pywt.waverec2(coeffs, 'haar')
    if denoised_image.shape != image.shape:
        denoised_image = resize(denoised_image, image.shape, mode='reflect', anti_aliasing=True)
    return denoised_image

def apply_total_variation_denoising(image):
    denoised_image = denoise_tv_chambolle(image, weight=0.1)
    return denoised_image

def process_single_nii(image_nii, mask_nii, view, slice_idx, target_size=(224,224)):
    """
    Bir NIfTI dosyasından istenen slice'ı alır,
    maske varsa uygular, çeşitli denoising/contrast işlemleri yapar
    ve 224x224 boyutunda döndürür.
    """
    img = image_nii.get_fdata()
    if mask_nii is not None:
        mask = mask_nii.get_fdata()
    else:
        mask = np.zeros_like(img)

    if view == 'sagittal':
        max_slices = img.shape[0]
        idx = min(slice_idx, max_slices-1)
        img_slice = img[idx, :, :]
        mask_slice = mask[idx, :, :]
    elif view == 'coronal':
        max_slices = img.shape[1]
        idx = min(slice_idx, max_slices-1)
        img_slice = img[:, idx, :]
        mask_slice = mask[:, idx, :]
    else:  # 'axial'
        max_slices = img.shape[2]
        idx = min(slice_idx, max_slices-1)
        img_slice = img[:, :, idx]
        mask_slice = mask[:, :, idx]

    img_slice = resize(img_slice, target_size, mode='reflect', anti_aliasing=True)
    mask_slice = resize(mask_slice, target_size, mode='reflect', anti_aliasing=False,
                        order=0, preserve_range=True)

    # Denoise + kontrast
    img_slice = enhance_contrast(img_slice)
    img_slice = apply_nlm_denoising(img_slice)
    img_slice = apply_wavelet_denoising(img_slice)
    img_slice = apply_total_variation_denoising(img_slice)

    mask_slice = (mask_slice > 0.05).astype(np.float32)

    highlight_factor = 1.8
    darken_factor = 0.8
    img_slice_masked = img_slice * mask_slice * highlight_factor
    img_slice_non_masked = img_slice * (1 - mask_slice) * darken_factor
    img_slice = img_slice_masked + img_slice_non_masked
    img_slice = np.clip(img_slice, 0, 1).astype(np.float32)

    return img_slice, mask_slice

def process_tumor_image(image_file, mask_file, view, slice_idx, token):
    """
    Meme kanseri segmentasyonu.
    'token' != "AUTHORIZED" ise erişim yok.
    """
    if token != "AUTHORIZED":
        return "Yetkisiz Erişim!", "Lütfen önce yüz tanıma ile giriş yapın."

    try:
        if not image_file:
            return "Image file yok", "Maske yok"

        image_nii = nib.load(image_file.name)
        mask_nii = nib.load(mask_file.name) if (mask_file and mask_file.name) else None

        img_slice, mask_slice = process_single_nii(
            image_nii, mask_nii, view, slice_idx, target_size=(224,224)
        )

        # Model input hazırlama (BxCxHxW)
        img_tensor = torch.from_numpy(img_slice).unsqueeze(0).unsqueeze(0)  # 1x1x224x224
        img_tensor = img_tensor.repeat(1, 3, 1, 1).to(device)              # 1x3x224x224

        with torch.no_grad():
            tumor_model.eval()
            logits = tumor_model(img_tensor)
            pred_prob = torch.sigmoid(logits)
            pred_mask = (pred_prob > 0.5).float()

        pred_mask_np = pred_mask.cpu().squeeze().numpy()

        return (img_slice, pred_mask_np)
    except Exception as e:
        return f"Hata: {str(e)}", "Hata"

def process_pulmonary_image(image, token):
    """
    Pulmoner Hipertansiyon sınıflandırması.
    'token' != "AUTHORIZED" ise erişim yok.
    """
    if token != "AUTHORIZED":
        return "Yetkisiz Erişim!", "0.00%"

    try:
        if image is None:
            return "Image yok", "0.00%"

        # NumPy -> PIL
        if isinstance(image, np.ndarray):
            if image.max() > 1.0:
                image = image / 255.0
            image = (image * 255).astype(np.uint8)
            image = Image.fromarray(image)

        # Gri ise 3 kanala çıkar vs.
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

        img_tensor = transform(image)
        if img_tensor.shape[0] == 1:
            img_tensor = img_tensor.repeat(3,1,1)

        img_tensor = img_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            pulmonary_model.eval()
            outputs = pulmonary_model(img_tensor)
            probs = torch.softmax(outputs, dim=1)
            pred_class = torch.argmax(probs, dim=1).item()
            confidence = probs[0][pred_class].item()

        class_names = ["Control", "Group1", "Group2", "Group3"]
        return class_names[pred_class], f"{confidence:.2%}"
    except Exception as e:
        return f"Hata: {str(e)}", "0.00%"

# ---------------------------------------------------
# 3) Gradio Arayüzü
# ---------------------------------------------------
with gr.Blocks() as demo:
    # Gradio'da session_state tutmak için:
    state = gr.State({"token": None})

    gr.Markdown("# Tüm Uygulama: Yüz Tanıma + Medikal Analiz")

    with gr.Tabs():
        # ---------------------------
        # A) LOGIN (Yüz Tanıma) Sekmesi
        # ---------------------------
        with gr.Tab("Login (Yüz Tanıma)"):
            gr.Markdown("Bu sekmede yüz tanıma yaparak sisteme giriş yapabilirsiniz.")
            face_input = gr.Image(type="numpy", label="Kamera Görüntüsü veya Foto")
            login_btn = gr.Button("Login")
            login_output_img = gr.Image(type="numpy", label="Sonuç Görseli")
            login_msg = gr.Text(label="Durum Mesajı")

            def login_click_fn(img, st):
                out_img, out_msg, st = login_with_face(img, st)
                return out_img, out_msg, st

            login_btn.click(
                fn=login_click_fn,
                inputs=[face_input, state],
                outputs=[login_output_img, login_msg, state]
            )

        # ---------------------------
        # B) Yeni Yüz Ekle Sekmesi
        # ---------------------------
        with gr.Tab("Yeni Yüz Ekle"):
            gr.Markdown("Sisteme yeni bir kullanıcı eklemek için")
            new_name = gr.Text(label="İsim giriniz")
            new_face_img = gr.Image(type="numpy", label="Yüz Fotoğrafı")
            add_btn = gr.Button("Yüzü Ekle")
            add_output_img = gr.Image(type="numpy", label="Eklenen Yüz")
            add_msg = gr.Text(label="Kayıt Durumu")

            add_btn.click(
                fn=add_new_face,
                inputs=[new_name, new_face_img],
                outputs=[add_output_img, add_msg]
            )

        # ---------------------------
        # C) Meme Kanseri Segmentasyonu Sekmesi
        # ---------------------------
        with gr.Tab("Breast Cancer Segmentation"):
            gr.Markdown("NIfTI formatında meme görüntüsünü ve opsiyonel maske dosyasını yükleyin.")

            nii_file = gr.File(label="NIfTI Image (.nii/.nii.gz)")
            mask_file = gr.File(label="Mask File (Opsiyonel)")
            view_choice = gr.Dropdown(["sagittal", "coronal", "axial"], value="axial", label="View Axis")
            slice_choice = gr.Slider(minimum=0, maximum=200, step=1, value=50, label="Slice Index")
            seg_button = gr.Button("Segmentasyonu Başlat")

            seg_output_img = gr.Image(label="İşlenmiş Slice", type="numpy")
            seg_output_mask = gr.Image(label="Segmentasyon Maskesi", type="numpy")

            def tumor_seg_fn(_nii, _mask, _view, _slice, st):
                return process_tumor_image(_nii, _mask, _view, _slice, st["token"])

            seg_button.click(
                fn=tumor_seg_fn,
                inputs=[nii_file, mask_file, view_choice, slice_choice, state],
                outputs=[seg_output_img, seg_output_mask]
            )

        # ---------------------------
        # D) Pulmoner Hipertansiyon Sekmesi
        # ---------------------------
        with gr.Tab("Pulmonary Hypertension Analysis"):
            gr.Markdown("Tomografi görüntüsü (PNG/JPG) yükleyerek PH sınıflandırması alın.")
            ph_img_input = gr.Image(type="numpy", label="Tomography Image")
            ph_button = gr.Button("PH Analizi")
            ph_pred = gr.Text(label="Prediction")
            ph_conf = gr.Text(label="Confidence")

            def ph_analysis_fn(img, st):
                return process_pulmonary_image(img, st["token"])

            ph_button.click(
                fn=ph_analysis_fn,
                inputs=[ph_img_input, state],
                outputs=[ph_pred, ph_conf]
            )

    # Uygulama sonu
demo.launch(debug=True, share=True)


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.1 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Kullanılan cihaz: cuda
MTCNN model yüklendi.
ArcFace ONNX modeli yüklendi.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


...

Kayıtlı yüz bulunamadı. Yeni bir sözlük oluşturuluyor...


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Breast Cancer Segmentation model loaded successfully.


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 180MB/s]


Pulmonary Hypertension Analysis model loaded successfully.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://90616d2394ceac0f3e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
